In [ ]:
# 라이브러리 설치
!pip install -q -U google-generativeai

In [ ]:
import pathlib
import textwrap

import google.generativeai as genai

# Used to securely store your API key
from google.colab import userdata

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
GOOGLE_API_KEY = 'Gemini_API_KEY'
genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
model = genai.GenerativeModel('gemini-pro')

### 텍스트 질문

In [ ]:
# 질문과 답변
request_str = input("Gemini에게 할 질문을 입력하세요: ")
response = model.generate_conten(request_str)

##sleep 이용 or 다른 방법으로 쉬게 만드셈
to_markdown(response.text)

### 이미지 질문

In [ ]:
# 이미지
import PIL.Image

img = PIL.Image.open('image.jpg')
model = genai.GenerativeModel('gemini-pro-vision')

#이미지 넘겨주기
response = model.generate_content(img)

# 질문과 답변
request_str = input("Gemini에게 할 질문을 입력하세요: ")
response = model.generate_conten(request_str)

##sleep 이용 or 다른 방법으로 쉬게 만드셈
to_markdown(response.text)

### chat 형태

In [ ]:
model = genai.GenerativeModel('gemini-pro')
chat = model.start_chat(history=[])
chat

In [ ]:
# 질문과 답변
request_str = input("Gemini에게 할 질문을 입력하세요: ")
response = chat.send_message(request_str)

##sleep 이용 or 다른 방법으로 쉬게 만드문
to_markdown(response.text)

In [ ]:
# history로 출력
for message in chat.history:
  display(to_markdown(f'**{message.role}**: {message.parts[0].text}'))

# RAG 시도

In [ ]:
#!pip install chromadb tk

In [ ]:
import google.generativeai as genai
import chromadb
from chromadb.utils import embedding_functions
import tk

# Gemini API 키 설정
genai.configure(api_key="") ##키 넣어야함.

In [ ]:
# ChromaDB 설정 및 데이터 저장
def setup_chroma(documents):
    embedding_function = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="all-MiniLM-L6-v2")
    chroma_client = chromadb.Client()
    collection = chroma_client.create_collection(name="my_collection4", embedding_function=embedding_function)
    collection.add(documents=documents, ids=[f"id{i}" for i in range(len(documents))])
    return collection

# 관련 문서 검색
def retrieve_relevant_documents(query, collection, n_results=3):
    results = collection.query(query_texts=[query], n_results=n_results)
    return results["documents"][0]

# Gemini 채팅 모델 초기화
def initialize_gemini_chat():
    model = genai.GenerativeModel('gemini-1.5-pro')
    chat = model.start_chat(history=[])
    return chat

# 답변 생성 및 채팅 기록 업데이트
def throw_chat_context(chat, relevant_documents):
    prompt = f"""
    다음 context를 참고하여 질문에 답변하라.
    문맥: {relevant_documents}
    """
    response = chat.send_message(prompt)

# 이미지 파일을 base64로 인코딩
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

def generate_chat_response(chat, query, image_path=None):
    prompt = f"""
    chat의 context를 참고하여 질문에 답변하세요.
    질문: {query}
    답변:
    """
    if image_path :
        image_data = encode_image(image_path)
        response = chat.send_message(
            [prompt, {"mime_type":"image/jpeg", "data":image_data}]
        )
    else :
        response = chat.send_message(prompt)
    return response.text

def select_image():
    root = tk.Tk()
    root.withdraw()
    file_path = filedialog.askopenfilename(filetypes=[("Image files", "*.jpg *.jpeg *.png")])
    return file_path

In [ ]:
# 예시 데이터-폴더에서 .txt 파일 읽어오기
import os
def load_documents_from_folder(folder_path):
    documents = []
    filenames = []

    for filename in os.listdir(folder_path):
        if filename.endswith(".txt"):  # .txt 파일만 로드
            with open(os.path.join(folder_path, filename), "r", encoding="utf-8") as file:
                content = file.read().strip()
                documents.append(content)
                filenames.append(filename)

    return documents, filenames

folder_path = "./doc_folder"
documents = load_documents_from_folder(folder_path)

# print(documents[0][0])

In [ ]:
# ChromaDB 설정
collection = setup_chroma(documents[0])

# Gemini 채팅 모델 초기화
chat = initialize_gemini_chat()

In [ ]:
# 사용자 질의 및 답변 생성
user_query = input("질문의 주제가 무엇인가요?: ")
relevant_docs = retrieve_relevant_documents(user_query, collection)
throw_chat_context(chat, relevant_docs)

while True:
    user_query = input("질문: ")
    if user_query.lower() == "종료":
        break
    if user_query.lower() == "이미지 입력":
        print("이미지 입력은 아직 미완성")
        '''
        image_path = select_image()
        if image_path:
            user_query = input("질문: ")
            answer = generate_chat_response(chat, user_query, image_path)
        else:
            print("이미지 선택이 취소되었습니다.")
            continue
        '''
    else:
        answer = generate_chat_response(chat, user_query)
    print("답변:", answer)

In [ ]:
# 모델 종류-예시 코드에서 업데이트가 있는 경우 바꿔야함.
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro-vision-latest
models/gemini-pro-vision
models/gemini-1.5-pro-latest
models/gemini-1.5-pro-001
models/gemini-1.5-pro-002
models/gemini-1.5-pro
models/gemini-1.5-flash-latest
models/gemini-1.5-flash-001
models/gemini-1.5-flash-001-tuning
models/gemini-1.5-flash
models/gemini-1.5-flash-002
models/gemini-1.5-flash-8b
models/gemini-1.5-flash-8b-001
models/gemini-1.5-flash-8b-latest
models/gemini-1.5-flash-8b-exp-0827
models/gemini-1.5-flash-8b-exp-0924
models/gemini-2.5-pro-exp-03-25
models/gemini-2.0-flash-exp
models/gemini-2.0-flash
models/gemini-2.0-flash-001
models/gemini-2.0-flash-exp-image-generation
models/gemini-2.0-flash-lite-001
models/gemini-2.0-flash-lite
models/gemini-2.0-flash-lite-preview-02-05
models/gemini-2.0-flash-lite-preview
models/gemini-2.0-pro-exp
models/gemini-2.0-pro-exp-02-05
models/gemini-exp-1206
models/gemini-2.0-flash-thinking-exp-01-21
models/gemini-2.0-flash-thinking-exp
models/gemini-2.0-flash-thinking-exp-1219
models/learnlm-1.5-pro-e